In [125]:
import pandas as pd
import numpy as np

data = pd.read_excel("./processed_quant_mid.xlsx")
data.set_index('Date', inplace=True)

#  상장폐지 수익률 -100% 처리를 위한 데이터 전처리
#  데이터 각 열의 마지막 유효값 인덱스 확인 및 마지막 유효값 다음 값은 0으로 처리
last_valid_index = data.apply(lambda col: col.last_valid_index())
for column in data.columns:
    last_valid_date = data[column].last_valid_index()
    next_row_index = data.index.get_loc(last_valid_date) + 1 if last_valid_date is not None else None
    if next_row_index is not None and next_row_index < len(data):
        data.at[data.index[next_row_index], column] = 0


In [126]:
monthly_return = data.pct_change(1)

/var/folders/dd/cw0ctdzd5zg5s2r6dn4104fr0000gn/T/ipykernel_93664/3789134966.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  monthly_return = data.pct_change(1)


In [127]:

# J, K 기간 리스트 생성
J = [3,6,9,12] 
K = [3,6,9,12]

# 각 종목 과거 J개월의 수익률을 저장할 딕셔너리
lookback_returns = {}

# for period in J:
#     lookback_returns[period] = data.pct_change(period)

for period in J:
    lookback_returns[period] = (1 + monthly_return).rolling(window=period).apply(np.prod, raw=True) - 1

In [128]:
#포트폴리오 수익률을 저장할 딕셔너리
buy_portfolio_returns = {}
sell_portfolio_returns = {}


# J-K 조합에 따른 buy, sell 포트폴리오 수익률 저장
for j in J:
    for k in K:
        buy_portfolio_returns[(j, k)] = [] 
        sell_portfolio_returns[(j, k)] = []

# 상위 30%, 하위 30% 종목 에 대한 Holding period 수익률 구하는 함수 선언
# def calculate_future_returns(data, past_returns, holding_period):
#     future_returns = data.pct_change(holding_period)
#     top_30_returns = future_returns[past_returns >= past_returns.quantile(0.7)].mean(axis=1)
#     bottom_30_returns = future_returns[past_returns <= past_returns.quantile(0.3)].mean(axis=1)
#     return top_30_returns, bottom_30_returns
        
def calculate_future_returns(data, past_returns, holding_period):
    future_returns = data.rolling(window=holding_period).mean()
    top_30_returns = future_returns[past_returns >= past_returns.quantile(0.7)].mean(axis=1)
    bottom_30_returns = future_returns[past_returns <= past_returns.quantile(0.3)].mean(axis=1)
    return top_30_returns, bottom_30_returns

# J,K조합에 따른 수익률을 저장하기 위한 리스트 생성
results = []

# J,K 조합에 따른 Holding period 수익률 계산 및 buy, sell 포트폴리오별로 result리스트에 저장
for j in J:
    for k in K:
        top_30_returns, bottom_30_returns = calculate_future_returns(monthly_return, lookback_returns[j], k)
        buy_portfolio_returns[(j, k)].append(top_30_returns)
        sell_portfolio_returns[(j, k)].append(bottom_30_returns)

        buy_returns_df = pd.DataFrame(buy_portfolio_returns[(j, k)]).T.mean(axis=1)
        sell_returns_df = pd.DataFrame(sell_portfolio_returns[(j, k)]).T.mean(axis=1)
        buy_avg_return = buy_returns_df.mean()
        sell_avg_return = sell_returns_df.mean()
        buy_minus_sell_avg_return = buy_avg_return - sell_avg_return

        if buy_avg_return:
            results.append({'J': j, 'K': k, 'Portfolio': 'buy', 'Return': buy_avg_return})
        if sell_avg_return:
            results.append({'J': j, 'K': k, 'Portfolio': 'sell', 'Return': sell_avg_return})
        if buy_minus_sell_avg_return : 
            results.append({'J': j, 'K': k, 'Portfolio': 'buy-sell', 'Return': buy_minus_sell_avg_return})
        # results.append({'J': j, 'K': k, 'Buy_Avg_Return': buy_avg_return, 'Sell_Avg_Return': sell_avg_return})

        # print(f'{j,k}: {buy_avg_return} // {sell_avg_return}')
            


In [129]:
#results 데이터프레임으로 변환 및 결과 출력
df_results = pd.DataFrame(results)

# df_results
df_results.pivot_table(index = ['J', 'Portfolio' ],  columns='K')

Return                              
K                   3         6         9         12
J  Portfolio                                        
3  buy        0.121984  0.066596  0.049441  0.041339
   buy-sell   0.204430  0.091474  0.058626  0.043636
   sell      -0.082446 -0.024879 -0.009185 -0.002297
6  buy        0.089693  0.090866  0.064235  0.052045
   buy-sell   0.142064  0.144070  0.088682  0.064758
   sell      -0.052371 -0.053205 -0.024447 -0.012714
9  buy        0.077069  0.075302  0.076427  0.060410
   buy-sell   0.117425  0.113364  0.116222  0.082741
   sell      -0.040356 -0.038062 -0.039795 -0.022331
12 buy        0.070264  0.068290  0.067586  0.068426
   buy-sell   0.106203  0.100020  0.098441  0.100524
   sell      -0.035939 -0.031729 -0.030855 -0.032097